In [ ]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#Uses gpu, if thats not possible it makes use of the cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Resizes the images to 128x128 pixels, Turns them into a tensor, and normalizes them
tf = [
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
]

#Data Loaders(Training Data, Testing Data)
train_dl = DataLoader(
    datasets.ImageFolder
)

